# JCC2 Data Analysis Notebook
This notebook demonstrates how to use the JCC2 Data Processor for analyzing questionnaire and data collection formats.

## 1. Setup and Imports

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json

# Configure visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Configure pandas display
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)

In [ ]:
# Import JCC2 processor
# Option 1: If the file is in the same directory
from jcc2_data_processor import create_processor, DataFormat

# Option 2: If the file is in a different directory
# import sys
# sys.path.append('/path/to/jcc2_processor')
# from jcc2_data_processor import create_processor, DataFormat

print("JCC2 Processor imported successfully!")

## 2. Load and Explore Data

In [ ]:
# Specify your data file
# Change this to your actual file path
csv_file = "mock_20_jcc2_user_questionnaire.csv"

# Create processor (automatically detects format)
processor = create_processor(csv_file)
print(f"Detected format: {processor.format_type.value}")

# Load data
df = processor.load_data()
print(f"\nLoaded {len(df)} rows and {len(df.columns)} columns")
print(f"Found {len(processor.sections)} sections")
print(f"System columns: {processor.system_columns}")

In [ ]:
# Display basic information
display(df.head())

In [ ]:
# Show available sections
print("Available sections:")
for i, (section, fields) in enumerate(processor.sections.items()):
    print(f"{i+1}. {section}: {len(fields)} fields")
    if i >= 9:  # Show first 10
        print("...")
        break

## 3. Data Validation

In [ ]:
# Validate data
errors = processor.validate_data()
print(f"Found {len(errors)} validation errors")

if errors:
    # Show first few errors
    errors_df = pd.DataFrame(errors[:10])
    display(errors_df)

## 4. Section Analysis

In [ ]:
# Analyze a specific section
section_name = list(processor.sections.keys())[0]
section_summary = processor.get_section_summary(section_name)

print(f"Section: {section_name}")
print(f"Total fields: {section_summary['total_fields']}")

# Create summary DataFrame
field_stats = []
for field, stats in section_summary['field_summaries'].items():
    field_stats.append({
        'Field': field.split('.')[-1],
        'Type': stats['field_type'],
        'Completion Rate': f"{stats['completion_rate']:.1%}",
        'Non-null Count': stats['non_null_count']
    })

stats_df = pd.DataFrame(field_stats)
display(stats_df)

## 5. Format-Specific Analysis

In [ ]:
# Get format-specific summary
format_summary = processor.get_format_specific_summary()

if processor.format_type == DataFormat.USER_QUESTIONNAIRE:
    print("=== User Questionnaire Analysis ===")
    print(f"Effectiveness ratings analyzed: {len(format_summary.get('effectiveness_ratings', {}))}")
    print(f"Frequency distributions analyzed: {len(format_summary.get('frequency_distributions', {}))}")
    
    # Show section completion rates
    if 'section_completion_rates' in format_summary:
        completion_df = pd.DataFrame(
            [(k, f"{v:.1%}") for k, v in format_summary['section_completion_rates'].items()],
            columns=['Section', 'Completion Rate']
        )
        display(completion_df.head(10))
        
elif processor.format_type == DataFormat.DATA_COLLECTION:
    print("=== Data Collection Analysis ===")
    print(f"Task performance metrics: {len(format_summary.get('task_performance_metrics', {}))}")
    print(f"Workaround analysis: {len(format_summary.get('workaround_analysis', {}))}")
    print(f"Datatable summaries: {len(format_summary.get('datatable_summaries', {}))}")

## 6. Visualizations

In [ ]:
# Prepare visualization data
viz_data = processor.prepare_visualization_data()
format_viz = processor.prepare_format_specific_visualizations()

print("Available visualizations:")
for key in viz_data.keys():
    print(f"- {key}")
for key in format_viz.keys():
    print(f"- {key} (format-specific)")

In [ ]:
# Application usage visualization
if 'application_usage' in viz_data:
    app_usage_df = viz_data['application_usage']
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Total fields per application
    app_usage_df.sort_values('total_fields', ascending=True).plot(
        x='application', y='total_fields', kind='barh', ax=ax1, legend=False
    )
    ax1.set_xlabel('Total Fields')
    ax1.set_title('Fields per Application')
    
    # Average responses per application
    app_usage_df.sort_values('avg_responses', ascending=True).plot(
        x='application', y='avg_responses', kind='barh', ax=ax2, legend=False, color='orange'
    )
    ax2.set_xlabel('Average Responses')
    ax2.set_title('Average Responses per Application')
    
    plt.tight_layout()
    plt.show()

### 6.1 Format-Specific Visualizations

In [ ]:
# For User Questionnaire: Effectiveness Heatmap
if processor.format_type == DataFormat.USER_QUESTIONNAIRE and 'effectiveness_heatmap' in format_viz:
    effectiveness_df = format_viz['effectiveness_heatmap']
    
    # Sample 10 columns for visualization
    sample_cols = effectiveness_df.columns[:10]
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(
        effectiveness_df[sample_cols].T, 
        cmap='RdYlGn', 
        center=3.5, 
        vmin=1, 
        vmax=6,
        cbar_kws={'label': 'Effectiveness Score'},
        yticklabels=[col.split('.')[-1][:30] for col in sample_cols]
    )
    plt.title('Effectiveness Ratings Heatmap (Sample)')
    plt.xlabel('Response ID')
    plt.tight_layout()
    plt.show()

In [ ]:
# For Data Collection: Task Performance
if processor.format_type == DataFormat.DATA_COLLECTION and 'task_performance' in format_viz:
    task_perf_df = format_viz['task_performance']
    
    plt.figure(figsize=(10, 8))
    task_perf_df.sort_values('success_rate').plot(
        x='task', 
        y='success_rate', 
        kind='barh',
        legend=False
    )
    
    # Add percentage labels
    for idx, row in task_perf_df.iterrows():
        plt.text(row['success_rate'], idx, f"{row['success_rate']:.1%}", 
                va='center', ha='left', fontsize=9)
    
    plt.xlabel('Success Rate')
    plt.title('Task Success Rates')
    plt.xlim(0, 1.1)
    plt.tight_layout()
    plt.show()

## 7. Export Results

In [ ]:
# Export summary to JSON
output_file = f"jcc2_{processor.format_type.value}_analysis.json"
summary = processor.export_summary(output_file)
print(f"Summary exported to {output_file}")

# Display summary structure
print("\nSummary structure:")
for key in summary.keys():
    if isinstance(summary[key], dict):
        print(f"- {key}: {len(summary[key])} items")
    else:
        print(f"- {key}: {type(summary[key]).__name__}")

In [ ]:
# Create Excel export with multiple sheets
excel_file = f"jcc2_{processor.format_type.value}_analysis.xlsx"

with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
    # Export raw data (first 100 rows as sample)
    df.head(100).to_excel(writer, sheet_name='Sample_Data', index=False)
    
    # Export section summaries
    section_stats = []
    for section in list(processor.sections.keys())[:5]:  # First 5 sections
        summary = processor.get_section_summary(section)
        section_stats.append({
            'Section': section,
            'Fields': summary['total_fields'],
            'Avg Completion': np.mean([f['completion_rate'] for f in summary['field_summaries'].values()])
        })
    
    pd.DataFrame(section_stats).to_excel(writer, sheet_name='Section_Summary', index=False)
    
    # Export validation errors
    if errors:
        pd.DataFrame(errors[:100]).to_excel(writer, sheet_name='Validation_Errors', index=False)

print(f"Excel analysis exported to {excel_file}")

## 8. Interactive Analysis Widget

In [ ]:
# Create interactive section explorer
try:
    import ipywidgets as widgets
    from IPython.display import display
    
    # Create dropdown for sections
    section_dropdown = widgets.Dropdown(
        options=list(processor.sections.keys()),
        description='Section:',
        style={'description_width': 'initial'}
    )
    
    output = widgets.Output()
    
    def explore_section(section):
        with output:
            output.clear_output()
            
            summary = processor.get_section_summary(section)
            print(f"Section: {section}")
            print(f"Total fields: {summary['total_fields']}")
            
            # Field type distribution
            type_counts = {}
            for field_data in summary['field_summaries'].values():
                field_type = field_data['field_type']
                type_counts[field_type] = type_counts.get(field_type, 0) + 1
            
            print("\nField types:")
            for ftype, count in type_counts.items():
                print(f"  - {ftype}: {count}")
            
            # Completion stats
            completion_rates = [f['completion_rate'] for f in summary['field_summaries'].values()]
            print(f"\nAverage completion rate: {np.mean(completion_rates):.1%}")
            print(f"Min completion rate: {np.min(completion_rates):.1%}")
            print(f"Max completion rate: {np.max(completion_rates):.1%}")
    
    # Create interactive widget
    widgets.interactive(explore_section, section=section_dropdown)
    display(section_dropdown)
    display(output)
    
    # Initialize with first section
    explore_section(section_dropdown.value)
    
except ImportError:
    print("ipywidgets not installed. Run: pip install ipywidgets")